Extrakce hodnot z rastru
---

In [ ]:
library(raster)

In [ ]:
dat_s<-stack(list.files(path="../../data/sentinel", pattern=".asc$|.tiff$",full.name=T)[c(4,8)])
ndvi<-(dat_s[[2]]-dat_s[[1]])/(dat_s[[2]]+dat_s[[1]])

In [ ]:
gps<-shapefile("../../data/gps_points.shp")

In [ ]:
plot(ndvi)
plot(gps,add=T)

In [ ]:
ndvi

In [ ]:
gps
gps@proj4string

In [ ]:
gps<-spTransform(gps, CRS(crs(ndvi)@projargs))

In [ ]:
gps
gps@proj4string

In [ ]:
crs(ndvi)@projargs==crs(gps)@projargs

In [ ]:
plot(ndvi)
plot(gps,add=T)

Funkce `raster::extract` získá k bodům hodnoty z rastru. Lze zvolit i buffer a funkci (např. mean). Pokud data nejsou ve stejném CRS funkce si s tím poradí

In [ ]:
?extract

In [ ]:
point_data<-extract(ndvi,gps,sp=T)
point_data

In [ ]:
point_data<-extract(ndvi,gps,sp=T,buffer=1000,fun=mean)

In [ ]:
point_data

Pokud chceme buffery zobrazit, nebo dál s ními pracovat, pro vytvoření objektu (polygony) s buffery použijeme funkci`rgeos::gBuffer`.

In [ ]:
library(rgeos)

In [ ]:
?gBuffer

In [ ]:
buff_gps<-gBuffer(gps,width=1000)

In [ ]:
plot(ndvi)
plot(buff_gps,add=T)

Ṕomocí buffere můžeme rastr ořezat:

In [ ]:
buff_mask <- mask(ndvi,buff_gps)

In [ ]:
plot(buff_mask)

Nebo vytvořit binární masku:

In [ ]:
buff_mask*0+1

In [ ]:
plot(buff_mask*0+1)

Funkce `raster::crop` ořeže rastr podle rozsahu vrstvy - extentu

In [ ]:
?crop

In [ ]:
plot(crop(ndvi,buff_gps,))

Pro změnu projekce rastru slouží `raster::projectRaster`, lze měnit CRS ale i rozlišení.

In [ ]:
ndvi_wgs<-projectRaster(ndvi,crs="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0")

In [ ]:
?projectRaster

In [ ]:
ndvi_wgs

Klasifikace obrazu
---------------------------

In [ ]:
raster_data_list<-list.files(path="../../data/sentinel", pattern=".tiff$",full.name=T)

In [ ]:
sent<-stack(raster_data_list)

In [ ]:
for (i in raster_data_list){
    print(res(raster(i)))
    }

In [ ]:
sent<-stack()
for (i in raster_data_list){
    i<-raster(i)
    i<-projectRaster(i,crs=crs(i),res=60)
    sent<-stack(sent,i)
    }

In [ ]:
sent

Nejjednodušší způsob klasifikace je tzv. neřízená klasifikace (funkce `RStoolbox::unsuperClass`)

In [ ]:
library(RStoolbox)

In [ ]:
?unsuperClass

In [ ]:
un_class<-unsuperClass(sent,nClasses = 4,nSamples = 10000, norm = T)

In [ ]:
plot(un_class$map)